# Sueño

## 1. Definición del Problema de Machine Learning

El objetivo del siguiente trabajo es predecir la calidad de sueño de una persona basándonos en datos como duración del sueño, niveles de estrés, nivel de actividad física durante el día (ya sea caminando o haciendo deporte), etc.

El target (calidad de sueño) consiste en un conjunto de valores discretos multiclase (marcados con valores que van del 1 al 5).

Las métricas principales de clasificación son: F1-score, precisión, recall o accuracy.

La métrica secundaria es la matriz de confusión para ver qué clases se confunden más.

**Impacto del proyecto**
- Si el modelo predice una mala calidad del sueño, el sector hospitalario podrá establecer medidas personalizadas acorde a los problemas de las personas.
- Tener un sueño reparador podría:
  1. Mejorar la satisfacción personal y laboral del usuario, reduciendo problemas de salud relacionados con el sueño y mejorando la productividad en su día a día.
  2. Evitar accidentes de tráfico, lo que puede ayudar a salvar vidas.

Los **requisitos mínimos** de rendimiento serán:
- El modelo debe alcanzar al menos un 80% de precisión en la exactitud de los datos (si es de 90% o más sería muy bueno) siempre y cuando las clases estén balanceadas.
- Los valores de F1-score deben tener un mínimo de 0.60 para ser aceptables, si tiene 0.70 serán buenos y si son mayores a 0.80 será excelente.
- La matriz de confusión será buena si no comete errores graves en las clases importantes.

## 2. Recolección y Exploración de Datos

1. Obtener el dataset completo
2. Realizar análisis exploratorio inicial:
   - Dimensiones del dataset
   - Tipos de datos
   - Estadísticas descriptivas básicas
   - Distribución de variables
   - Correlaciones
3. Verificar la calidad de los datos:
   - Identificar valores faltantes
   - Detectar inconsistencias
   - Verificar el balance de clases (para clasificación)
   - Identificar posibles errores en los datos

In [78]:
import warnings

#Tratamiento de Datos
import pandas as pd
import numpy as np

#Gráficos
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

#Machine Learning
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

In [79]:
sleep=pd.read_csv('../data/raw/Sleep.csv')

In [80]:
df=sleep.copy()

In [81]:
df.head()

,Gender,Age,Sleep duration,Sleep quality,Stress level,Blood pressure,Heart rate,Daily steps,Physical activity,Height,...,Smoking,Medical issue,Ongoing medication,Smart device before bed,Average screen time,Blue-light filter,Discomfort Eye-strain,Redness in eye,Itchiness/Irritation in eye,Dry Eye Disease
0,F,24,9.5,2,1,137/89,67,3000,31,161,...,N,Y,Y,N,8.7,N,Y,Y,N,Y
1,M,39,9.6,2,3,108/64,60,12000,74,164,...,N,Y,Y,N,9.6,Y,N,N,Y,Y
2,F,45,5.4,1,5,134/81,95,12000,93,179,...,N,N,N,Y,4.0,N,Y,N,N,N
3,F,45,5.4,4,5,110/90,78,19000,32,160,...,N,Y,N,N,7.6,N,Y,N,Y,N
4,F,42,5.7,3,2,99/67,72,4000,173,179,...,N,Y,N,N,3.5,N,Y,Y,N,Y


### Dimensiones del dataset

In [82]:
df.shape

(20000, 26)

### Tipos de datos

In [83]:
df.info()
print("Filas:", df.shape[0])
print("Columnas:", df.shape[1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Gender                       20000 non-null  object 
 1   Age                          20000 non-null  int64  
 2   Sleep duration               20000 non-null  float64
 3   Sleep quality                20000 non-null  int64  
 4   Stress level                 20000 non-null  int64  
 5   Blood pressure               20000 non-null  object 
 6   Heart rate                   20000 non-null  int64  
 7   Daily steps                  20000 non-null  int64  
 8   Physical activity            20000 non-null  int64  
 9   Height                       20000 non-null  int64  
 10  Weight                       20000 non-null  int64  
 11  Sleep disorder               20000 non-null  object 
 12  Wake up during night         20000 non-null  object 
 13  Feel sleepy duri

In [84]:
df.duplicated().sum()

np.int64(0)

In [85]:
df.isnull().sum()

Gender                         0
Age                            0
Sleep duration                 0
Sleep quality                  0
Stress level                   0
Blood pressure                 0
Heart rate                     0
Daily steps                    0
Physical activity              0
Height                         0
Weight                         0
Sleep disorder                 0
Wake up during night           0
Feel sleepy during day         0
Caffeine consumption           0
Alcohol consumption            0
Smoking                        0
Medical issue                  0
Ongoing medication             0
Smart device before bed        0
Average screen time            0
Blue-light filter              0
Discomfort Eye-strain          0
Redness in eye                 0
Itchiness/Irritation in eye    0
Dry Eye Disease                0
dtype: int64

### Valores únicos

In [96]:
df['Gender'].value_counts()

Gender
M    10028
F     9972
Name: count, dtype: int64

In [86]:
print(df['Gender'].unique())
print(df['Stress level'].unique())
print(df['Blood pressure'].unique())
print(df['Heart rate'].unique())
print(df['Sleep disorder'].unique())
print(df['Wake up during night'].unique())
print(df['Feel sleepy during day'].unique())
print(df['Caffeine consumption'].unique())
print(df['Alcohol consumption'].unique())
print(df['Smoking'].unique())
print(df['Medical issue'].unique())
print(df['Ongoing medication'].unique())
print(df['Smart device before bed'].unique())
print(df['Blue-light filter'].unique())

['F' 'M']
[1 3 5 2 4]
['137/89' '108/64' '134/81' ... '133/61' '125/72' '114/62']
[ 67  60  95  78  72  79  63  97  81  96  88  64  66  87  76  91  89  84
  98 100  68  62  99  82  71  77  73  65  94  85  61  74  80  83  69  92
  70  90  86  93  75]
['Y' 'N']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['Y' 'N']
['Y' 'N']
['N' 'Y']
['N' 'Y']


### Estadísticas descriptivas básicas

In [87]:
print(df.describe())

                Age  Sleep duration  Sleep quality  Stress level  \
count  20000.000000    20000.000000   20000.000000  20000.000000   
mean      31.422800        6.998245       2.997250      2.993750   
std        8.103717        1.731723       1.412283      1.407235   
min       18.000000        4.000000       1.000000      1.000000   
25%       24.000000        5.500000       2.000000      2.000000   
50%       31.000000        7.000000       3.000000      3.000000   
75%       39.000000        8.500000       4.000000      4.000000   
max       45.000000       10.000000       5.000000      5.000000   

         Heart rate   Daily steps  Physical activity        Height  \
count  20000.000000  20000.000000       20000.000000  20000.000000   
mean      79.912200  10536.900000          90.069750    174.865900   
std       11.808279   5752.729186          52.317283     14.719903   
min       60.000000   1000.000000           0.000000    150.000000   
25%       70.000000   6000.000000    

In [88]:
print(df.describe(include="all"))

       Gender           Age  Sleep duration  Sleep quality  Stress level  \
count   20000  20000.000000    20000.000000   20000.000000  20000.000000   
unique      2           NaN             NaN            NaN           NaN   
top         M           NaN             NaN            NaN           NaN   
freq    10028           NaN             NaN            NaN           NaN   
mean      NaN     31.422800        6.998245       2.997250      2.993750   
std       NaN      8.103717        1.731723       1.412283      1.407235   
min       NaN     18.000000        4.000000       1.000000      1.000000   
25%       NaN     24.000000        5.500000       2.000000      2.000000   
50%       NaN     31.000000        7.000000       3.000000      3.000000   
75%       NaN     39.000000        8.500000       4.000000      4.000000   
max       NaN     45.000000       10.000000       5.000000      5.000000   

       Blood pressure    Heart rate   Daily steps  Physical activity  \
count          

Se observa que hay más de 20000 filas de información y 26 columnas. Lo que es un volumen adecuado para realizar un estudio de machine learning.
La población se comprende de individuos entre 18 y 45 años, tanto hombres como mujeres que tienen características diferentes afectando su calidad de sueño.


## 3. Preprocesamiento de Datos Inicial

1. Eliminar duplicados si existen
2. Tratar valores faltantes:
   - Decidir entre eliminar o imputar
   - Documentar la estrategia elegida
3. Identificar y tratar outliers:
   - Análisis estadístico de outliers
   - Decidir estrategia (eliminar, transformar o mantener)
4. Limpieza general de datos

### Limpieza de Datos

## 4. Feature Engineering

1. Selección inicial de características:
   - Análisis de correlaciones
   - Importancia de variables
2. Creación de nuevas características:
   - Combinaciones de variables existentes
   - Transformaciones matemáticas
3. Aplicación de técnicas no supervisadas (si es necesario):
   - PCA para reducción de dimensionalidad
   - Clustering para nuevas features

### Eliminación de Features No Deseadas

In [89]:
df.drop(columns=['Discomfort Eye-strain', 'Redness in eye', 'Itchiness/Irritation in eye', 'Dry Eye Disease'], inplace=True)

### Gender

### Blood Pressure

In [90]:
# División de presión sanguínea entre sistólica y diastólica

df[['systolic', 'diastolic']] = df['Blood pressure'].str.split('/', expand=True).astype(int)
df

,Gender,Age,Sleep duration,Sleep quality,Stress level,Blood pressure,Heart rate,Daily steps,Physical activity,Height,...,Caffeine consumption,Alcohol consumption,Smoking,Medical issue,Ongoing medication,Smart device before bed,Average screen time,Blue-light filter,systolic,diastolic
0,F,24,9.5,2,1,137/89,67,3000,31,161,...,N,N,N,Y,Y,N,8.7,N,137,89
1,M,39,9.6,2,3,108/64,60,12000,74,164,...,N,Y,N,Y,Y,N,9.6,Y,108,64
2,F,45,5.4,1,5,134/81,95,12000,93,179,...,Y,Y,N,N,N,Y,4.0,N,134,81
3,F,45,5.4,4,5,110/90,78,19000,32,160,...,Y,N,N,Y,N,N,7.6,N,110,90
4,F,42,5.7,3,2,99/67,72,4000,173,179,...,N,N,N,Y,N,N,3.5,N,99,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,M,26,9.4,2,3,132/64,91,20000,88,200,...,N,N,Y,Y,Y,Y,3.8,N,132,64
19996,F,39,7.3,3,4,101/75,65,2000,53,195,...,N,Y,N,Y,Y,N,3.8,N,101,75
19997,F,20,8.0,5,3,124/63,78,10000,17,186,...,Y,Y,Y,Y,N,N,8.4,N,124,63
19998,M,38,4.5,3,3,136/70,60,3000,115,170,...,Y,Y,Y,N,Y,N,2.2,N,136,70


In [91]:
# Comprobación de valores máximos y mínimos presentes en el dataframe
print("syst_max:", df['systolic'].max())
print("syst_min:", df['systolic'].min())
print("dias_max:", df['diastolic'].max())
print("dias_min:", df['diastolic'].min())

syst_max: 140
syst_min: 90
dias_max: 90
dias_min: 60


Los valores de presión sanguínea son los siguientes:
- **Low:** <=90 and <=60
- **Normal:** 120 and 80
- **Elevated:** 120-129 and <80
- **High/Hypertension:** 130-139 or 80-89
- **High/Hypertension 2:** >=140 or >=90

In [92]:
def blood_pressure(systolic, diastolic):
    if systolic >= 140 or diastolic >= 90:
        return "Hypertension 2"
    elif systolic >= 130 or diastolic >= 80:
        return "Hypertension 1"
    elif systolic >= 120 and diastolic < 80:
        return "Elevated"
    elif systolic > 90 and diastolic > 60:
        return "Normal"
    else:
        return "Low"

df["Blood Pressure"] = df.apply(lambda row: blood_pressure(row["systolic"], row["diastolic"]), axis=1)

In [93]:
df.head()

,Gender,Age,Sleep duration,Sleep quality,Stress level,Blood pressure,Heart rate,Daily steps,Physical activity,Height,...,Alcohol consumption,Smoking,Medical issue,Ongoing medication,Smart device before bed,Average screen time,Blue-light filter,systolic,diastolic,Blood Pressure
0,F,24,9.5,2,1,137/89,67,3000,31,161,...,N,N,Y,Y,N,8.7,N,137,89,Hypertension 1
1,M,39,9.6,2,3,108/64,60,12000,74,164,...,Y,N,Y,Y,N,9.6,Y,108,64,Normal
2,F,45,5.4,1,5,134/81,95,12000,93,179,...,Y,N,N,N,Y,4.0,N,134,81,Hypertension 1
3,F,45,5.4,4,5,110/90,78,19000,32,160,...,N,N,Y,N,N,7.6,N,110,90,Hypertension 2
4,F,42,5.7,3,2,99/67,72,4000,173,179,...,N,N,Y,N,N,3.5,N,99,67,Normal


In [94]:
# Eliminamos las columnas que ya no son necesarias
df= df.drop(['Blood pressure', 'systolic', 'diastolic'], axis=1)

In [95]:
df.head()

,Gender,Age,Sleep duration,Sleep quality,Stress level,Heart rate,Daily steps,Physical activity,Height,Weight,...,Feel sleepy during day,Caffeine consumption,Alcohol consumption,Smoking,Medical issue,Ongoing medication,Smart device before bed,Average screen time,Blue-light filter,Blood Pressure
0,F,24,9.5,2,1,67,3000,31,161,69,...,N,N,N,N,Y,Y,N,8.7,N,Hypertension 1
1,M,39,9.6,2,3,60,12000,74,164,87,...,N,N,Y,N,Y,Y,N,9.6,Y,Normal
2,F,45,5.4,1,5,95,12000,93,179,94,...,N,Y,Y,N,N,N,Y,4.0,N,Hypertension 1
3,F,45,5.4,4,5,78,19000,32,160,77,...,N,Y,N,N,Y,N,N,7.6,N,Hypertension 2
4,F,42,5.7,3,2,72,4000,173,179,99,...,N,N,N,N,Y,N,N,3.5,N,Normal


## 5. Preparación de Datos

1. Split de datos:
   - Train/Test/Validation
   - Mantener estratificación si es necesario
2. Procesamiento de variables (fit solo en train):
   - Codificación de variables categóricas
   - Normalización/Estandarización de variables numéricas
3. Balanceo de clases (solo en train si es necesario):
   - Oversampling
   - Undersampling
   - Técnicas híbridas

### Divide en train, test y validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Sleep quality', axis=1),
                                                    df['Sleep quality'],
                                                    test_size=0.2,
                                                    random_state=42)

### Target

In [ ]:
fig, axes = plt.subplots(1, 3, figsize= (15,5), sharey=True)

sns.countplot(x="Sleep quality", data=df, ax=axes[0])
axes[0].set_title("Original target")

sns.countplot(y_rus, ax=axes[1])
axes[1].set_title("Random Under Sampler")

sns.countplot(y_ros, ax=axes[2])
axes[2].set_title("Random Over Sampler")

## 6. Selección de Modelos Base

1. Implementar validación cruzada para varios modelos
2. Evaluar modelos básicos según el tipo de problema
3. Comparar resultados iniciales
4. Seleccionar los 3-5 mejores modelos
5. Documentar razones de selección

## 7. Optimización de Modelos Seleccionados

1. Realizar ajuste de hiperparámetros para cada modelo:
   - Grid Search
   - Random Search
   - Bayesian Optimization
2. Evaluar resultados de cada modelo optimizado
3. Considerar ensamble de mejores modelos
4. Seleccionar el modelo final

## 8. Evaluación y Análisis

1. Evaluar métricas finales en datos de test
2. Realizar análisis detallado de errores
3. Interpretar resultados:
   - Importancia de características
   - SHAP values
   - Partial dependence plots
4. Validar supuestos del modelo
5. Analizar casos específicos de error
6. Documentar hallazgos principales

## 9. Despliegue y Documentación

1. Preparar el modelo para producción:
   - Optimizar código
   - Crear pipeline de predicción
2. Documentar todo el proceso:
   - Decisiones tomadas
   - Resultados obtenidos
   - Limitaciones encontradas
3. Establecer plan de monitoreo:
   - Métricas a seguir
   - Frecuencia de actualización
   - Procedimientos de mantenimiento

## Análisis Bivariante

## Eliminación de Features

## Duplicados

## Missings

## Anomalías y Errores

## Outliers

## Feature Engineering

## Feature Reduction

## Escoger Métrica del Modelo

## Decidir sobre los Modelos

## Elegir Hiperparámetros

## Definición de Pipelines y Probamos

## Resultados